# Проверочное задание

# 🏃‍♂️🤸‍♂️🏋️‍♀️
Сегодня вы аналитики популярной сети фитнес-клубов, которая предлагает три вида тренировок для взрослых и детей:
- Обычное занятие в зале,
- Сайклинг (кардиотренировка на велотренажёре),
- Функциональная тренировка.

Первая тренировка в наших клубах всегда бесплатная.

Доступна следующая выгрузка данных:
- О наших клиентах — `users.xlsx`,

- Об оплатах тренировок — `payments.xlsx`,

- О посещении тренировок — `workouts.csv`.

Директор  сети и попросил ответить на вопросы, которые позволят более точно сформулировать планы на следующий год:

1. Как изменялась ежемесячная выручка по каждому типу тренировок в течение 2020 года?
2. Какие тренеры в сети провели больше всего платных занятий за все время?
3. Сколько денег и за какие типы тренировок принесли клиенты, которые начали заниматься в ноябре?
4. Какие регионы приносят нашей сети больше всего и меньше всего выручки? Результаты регионов будут обсуждаться на совете директоров, поэтому необходимо выгрузить этот отчет в файл формата Excel, чтобы включить его в раздаточный материал для совещания.

Выполнив следующие четыре задания, вы сможете последовательно ответить на все вопросы руководителя. Для начала убедитесь, что три новых файла с данными скачаны в папку с этим ноутбуком, импортируй нужные библиотеки и загрузи данные в датафреймы.

In [34]:
import pandas as pd
from datetime import datetime

users = pd.read_excel("users.xlsx") # загружаем данные по клиентам
payments = pd.read_excel("payments.xlsx") # загружаем данные по платежам
workouts = pd.read_csv("workouts.csv") # загружаем данные по тренировкам

pd.set_option('display.float_format', lambda x: '%.2f' % x) # эта строка поможет удобно выводить большие числа в этом ноутбуке

## Задание 1

Подготовь данные для ответа на вопрос: «__Как изменялась ежемесячная выручка по каждому типу тренировок в течение 2020 года?__».

### Как это сделать?

1. Изучи датафрейм `payments` и для начала переведи даты в формат `datetime`.

2. С помощью фильтрации датафрейма выбери платежи только за 2020 год.

3. Обрати внимание, что платежи хранятся в разных валютах — для удобства приведем все к рублям. Сначала добавь в датафрейм столбец `fx_rate` с курсом валюты и заполни его с помощью метода `apply()`: если это USD, то берем значение переменной `USD_RATE`, если RUR — ставим значение `1`. Предварительно нужно определить функцию, которая реализует эту несложную логику.

4. Добавь еще один столбец `amount_rub` с суммой платежа в рублях. Для этого умножь столбец с суммой платежа на столбец `fx_rate` с курсом валют.

5. Получи датафрейм с разбивкой выручки по месяцам и по типам тренировок с помощью группировки с суммой методом `groupby()`.

6. Выведи результат на экран.

🤔 *Предположи, основываясь на проделанном анализе.* Благодаря каким тренировкам компания могла получать больше выручки? У этого задания нет правильного ответа,  предлагается некое размышление.

In [35]:
payments['payment_date'] = pd.to_datetime(payments['payment_date'], format="%d.%m.%Y") #даты в формат datetime

In [36]:
payments_2020 = payments.loc[payments['payment_date'].dt.year == 2020].copy() #платежи только за 2020 год и создали независимую копию

In [37]:
USD_RATE = 73.14 # курс доллара
payments_2020['fx_rate'] = payments_2020['currency'].apply(lambda x: USD_RATE if x == 'USD' else 1)  # определи функцию для заполнения столбца fx_rate методом .apply()

In [38]:
amount_rub = payments_2020['amount'] * payments_2020['fx_rate'] #новый столбец с суммой платежа в рублях
payments_2020['amount_rub'] = amount_rub

In [39]:
groupby_payments_2020 = payments_2020.groupby(['workout_type', payments_2020['payment_date'].dt.month])['amount_rub'].sum() # сгруппируй данные по месяцам и по типам тренировок
gruopby_payments_2020_df = groupby_payments_2020.to_frame() #датафрейм
gruopby_payments_2020_df

amount_rub
workout_type    payment_date            
cycling         1               37611.51
                2               63026.64
                3               49843.52
                4             1006056.79
                5              103156.30
                6               55233.26
                7              119472.93
                8               84327.61
                9              752520.72
                10             886795.92
                11             184408.30
                12              38017.44
functional test 1                8000.52
                2                7020.81
                3               17098.39
                4                7015.99
                5                9050.01
                6                7020.81
                7                7800.90
                8                7110.81
                9               23597.35
                10              28857.61
general         1             9231431.45
                2            15056524.85
                3            19517875.66
                4            25308002.27
                5            27269168.04
                6            19536194.78
                7            21212408.89
                8            22034979.24
                9            23959753.45
                10           24389609.10
                11           24875912.84
                12            7924395.25

## Задание 2

Определи, __какой тренер в сети провел больше всего платных занятий за все время__. С самыми эффективными сотрудниками мы организуем интервью, чтобы понять, что влияет на их результат.

### Как это сделать?

1. Изучи датафрейм `workouts`.

2. Исключи из датафрейма тренеров из департамента `Sales`, так как они проводят только бесплатные тренировки, а нам интересны только платные занятия.

3. Сделай группировку по `trainer_id` и выведи количество уникальных тренировок по `workout_id`.

4. Найди максимальное количество тренировок и сохрани его в переменную `max_trainings`. Не забудь, что после группировки в прошлом пункте, индексом стал столбец `trainer_id`. Возможно, тебе понадобится метод, который возвращает индексы в колонки.

5. Выведи на экран id тренера, у которого количество тренировок равно значению `max_trainings`.



In [40]:
df = workouts.loc[workouts['trainer_department'] != 'Sales'].copy()
df

,workout_id,cost,start_at,status,workout_schedule_type,client_id,client_status,workout_type,trainer_department,trainer_id,currency
1,31123309,750.00,2020-11-21 13:00,success,regular,22034,new,general,Spartacus,940642,RUR
2,31412167,750.00,2020-11-28 13:00,success,regular,22034,new,general,Spartacus,940642,RUR
3,31703605,750.00,2020-12-05 13:00,success,regular,22034,active,general,Spartacus,940642,RUR
5,27205345,750.00,2020-08-15 10:00,success,single,88101,new,general,Consultant,10231664,RUR
6,27439942,750.00,2020-08-20 19:00,success,regular,88101,new,general,Consultant,10231664,RUR
...,...,...,...,...,...,...,...,...,...,...,...
104688,30976738,750.00,2020-11-19 08:00,success,single,4831015,active,general,Spartacus,6684752,RUR
104689,31118199,750.00,2020-11-21 11:00,success,regular,4831015,active,general,Spartacus,6684752,RUR
104690,31118238,750.00,2020-11-26 08:00,success,single,4831015,active,general,Spartacus,6684752,RUR
104691,31600293,750.00,2020-12-03 08:00,success,regular,4831015,active,general,Spartacus,6684752,RUR


In [41]:
#исключил из датафрейма тренеров из департамента Sales
workouts_paid = workouts[workouts['cost'].notna()].copy()

In [42]:
#группировка по trainer_id, количество уникальных тренировок по workout_id
trainer_workouts = workouts_paid.groupby('trainer_id')['workout_id'].nunique().reset_index()


In [43]:
#максимальное количество тренировок
max_trainings = trainer_workouts['workout_id'].max()

In [44]:
# id тренера, у которого количество тренировок равно значению max_trainings
top_trainer_id = trainer_workouts[trainer_workouts['workout_id'] == max_trainings]['trainer_id'].iloc[0]
print(f"Тренер с наибольшим количеством платных занятий: {top_trainer_id}")
print(f"Количество платных занятий: {max_trainings}")

Тренер с наибольшим количеством платных занятий: 7010157
Количество платных занятий: 307


## Задание 3

Посчитайте, __сколько денег и за какие типы тренировок принесли клиенты, которые начали заниматься в ноябре__.  

### Как это сделать?

1. Изучи датафрейм `users` и переведи даты в формат `datetime`.
2. Отбери только тех клиентов, кто начал заниматься с 1 по 30 ноября 2020 года, код уже готов, обрати внимание, как можно передать сразу 2 условия в `loc[]`
3. Объедини получившийся набор данных с платежами из датафрейма `payments` по `user_id`.

4. Сгруппируй результат по `workout_type` и найди сумму по `amount_rub`.

In [45]:
users

,first_contact_datetime,age,free_workout_datetime,first_payment_datetime,workouts_successful,workouts_total,months_active,user_id,region,geo_group
0,2020-08-08 13:57:25,NaN,2020-08-08 19:00:00,2020-08-08 21:18:34,10,13,2,2790000,NaN,СНГ
1,2020-05-02 00:30:02,NaN,2020-05-05 18:20:00,2020-05-05 18:40:29,26,38,7,780106,NaN,СНГ
2,2019-06-27 13:10:33,30.00,NaN,2020-03-18 16:06:43,27,33,6,1133376,NaN,СНГ
3,2020-04-22 15:37:58,22.00,2020-04-27 09:00:00,2020-04-27 09:41:37,59,59,8,1996499,NaN,СНГ
4,2016-06-23 16:21:40,24.00,2020-04-13 19:00:00,2020-04-13 19:16:51,8,12,2,57899,Москва и Московская область,Москва
...,...,...,...,...,...,...,...,...,...,...
4268,2019-01-23 14:37:51,28.00,2020-07-22 20:20:00,2020-07-22 21:00:00,25,27,4,3481612,Москва и Московская область,Москва
4269,2019-07-27 01:52:24,27.00,NaN,2020-02-28 10:15:01,44,66,10,1720022,Сахалинская область,Регионы РФ
4270,2017-04-24 11:00:29,NaN,NaN,2020-03-05 18:16:59,9,9,9,320154,Москва и Московская область,Москва
4271,2018-02-19 20:53:21,14.00,NaN,2020-02-05 20:03:14,100,101,11,1308708,Красноярский край,Регионы РФ


In [46]:
users['free_workout_datetime'] = pd.to_datetime(users['free_workout_datetime'], format='mixed', dayfirst=False) # переведи даты в формат datetime

In [47]:
# Отбери только тех клиентов, кто начал заниматься с 1 по 30 ноября 2020 года
nov_users = users.loc[(users['free_workout_datetime'].dt.month == 11) &
                      (users['free_workout_datetime'].dt.year == 2020)].copy()


In [48]:
USD_RATE = 73.14 # курс доллара
payments['fx_rate'] = payments['currency'].apply(lambda x: USD_RATE if x == 'USD' else 1)
payments['amount_rub'] = payments['amount'] * payments['fx_rate']






In [49]:
# объединение данных
nov_users_payments = pd.merge(nov_users, payments, on='user_id', how='inner')

In [50]:
#группировка данных
revenue_by_workout_type_nov_users = nov_users_payments.groupby('workout_type')['amount_rub'].sum().to_frame()
revenue_by_workout_type_nov_users

,amount_rub
workout_type,
cycling,77676.28
general,5636628.32


## Задание 4

Выясни, **какие регионы приносят нашей сети больше всего и меньше всего выручки**.

### Как это сделать

1. Создай новый датафрейм `user_regions` из колонок `user_id` и `region` датафрейма `users`.

2. Создай новый датафрейм `user_payments` из колонок `user_id` и `amount_rub` датафрейма `payments`.
3. C помощью `.merge()` объедини датафреймы `user_regions` и `user_payments`
4. Обрати внимание, что у некоторых клиентов не заполнен регион. `dropna()` — хороший метод для очистки датасета от строк, в которых не хватает каких-то значений.
5. Сгруппируй данные о выручке по региону, а затем отсортируй их.
6. Выведи на экран топ-5 регионов с самой маленькой выручкой.
7. Выведи на экран топ-5 регионов с самой большой выручкой.
8. Сохрани отсортированный по убыванию список регионов с выручкой в файл `task_fitnes.xlsx` с помощью метода `to_excel()`.

🤔 *Предположи, основываясь на проделанном анализе.* Как сейчас распределялась
выручка между регионами? У этого задания нет правильного ответа, можно поразмышлять.



In [51]:
#датафрейм user_regions из колонок user_id и region датафрейма users.
user_regions = users[['user_id', 'region']].copy()


In [52]:
#новый датафрейм user_payments из колонок user_id и amount_rub датафрейма payments.
user_payments = payments[['user_id', 'amount_rub']].copy()

In [53]:
#С помощью .merge() объединил датафреймы user_regions и user_payments
regional_revenue = pd.merge(user_regions, user_payments, on='user_id', how='inner')

In [54]:
#Очистка от пустых значений
regional_revenue.dropna(subset=['region'], inplace=True)

In [55]:
#группировка и сортировка данных о выручке по региона
revenue_by_region = regional_revenue.groupby('region')['amount_rub'].sum().sort_values(ascending=False).to_frame()


In [56]:
#Вывел на экран топ-5 регионов с самой маленькой выручкой.
print("Топ-5 регионов с самой маленькой выручкой:")
revenue_by_region.tail(5)

Топ-5 регионов с самой маленькой выручкой:


,amount_rub
region,
Республика Алтай,20172.72
Луганская область,19703.92
Еврейская АО,19615.55
Республика Калмыкия,9277.15
Магаданская область,8938.74


In [57]:
#Вывел на экран топ-5 регионов с самой большой выручкой.
print("Топ-5 регионов с самой большой выручкой:")
revenue_by_region.head(5)

Топ-5 регионов с самой большой выручкой:


,amount_rub
region,
Москва и Московская область,121137445.26
Санкт-Петербург и Ленинградская область,26429453.52
Новосибирская область,4968301.32
Краснодарский край,4061682.64
Республика Татарстан,3891956.28


In [58]:
#сохранил в формате excel.
revenue_by_region.to_excel("task_fitnes.xlsx")

# 🎉🎉🎉  